## 서울특별시 다산콜센터(☎120)의 주요 민원
* 서울특별시 다산콜센터(☎120)의 주요 민원(자주 묻는 질문)에 대한 답변정보
* https://opengov.seoul.go.kr/civilappeal/list

In [55]:
# 필요한 라이브러리들을 불러옵니다.
# pandas : 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구입니다.
# requests : 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옵니다.
# BeautifulSoup : request로 가져온 웹사이트의 html 태그를 찾기 위해 사용합니다.
# time : 한 번에 많은 양의 데이터를 가져오게 되면 서버에서 부담을주기 때문에 시간 간격을 두고 가져오기 위해 사용합니다.
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time

In [56]:
# 수집해온 파일을 읽어옵니다.
df = pd.read_csv("seoul-120-list.csv")
df.shape

(2645, 5)

## 일부 데이터를 우선 수집
* 현업에서는 모든 데이터를 수집해야 하지만 모든 데이터 수집시 서버에 부담이 될 수 있으며 오래 걸립니다.
* 또 동작하지 않거나 잘못 수집을 했다면 처음부터 다시 수집해야 하기 때문에 일부 데이터만을 통해 먼저 수집해 보는 것을 추천합니다.
* 일부 데이터를 먼저 수집해 보고 데이터가 필요할 때 전체를 수집해 봅니다.

In [57]:
# 30 개의 데이터를 먼저 수집해 봅니다.
df = df.head(30)
df.shape

(30, 5)

In [24]:
# 데이터의 일부를 head 로 가져옵니다.
df.head()

,번호,제목,생산일,조회수,view_no
0,2645,아빠 육아휴직 장려금,2021-08-17,144,23522464
1,2644,[서울산업진흥원] 서울메이드란?,2021-06-29,166,23194045
2,2643,(강북구) 정비중,2021-06-02,215,23032485
3,2642,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,288,22904492
4,2641,마포 뇌병변장애인 비전센터,2021-03-12,441,22477798


## 특정 내용 읽어오기
* 내용의 URL을 확인합니다.
* requests를 통해 내용에 접근합니다.
* BeautifulSoup의 select를 사용해 내용이 있는 태그를 찾습니다.


In [27]:
url = "https://opengov.seoul.go.kr/civilappeal/view/?nid=23522464"
print(url)
response = requests.get(url)
html = bs(response.text, "lxml")
html.select("div.line-all")[0].get_text()

https://opengov.seoul.go.kr/civilappeal/view/?nid=23522464


'\n\xa0     업무개요  \xa0남성근로자의 육아휴직을 장려하고 양육에 따른 경제적 부담을 완화함으로써 일과   가정생활의 양립 및 가족친화적인\xa0 \xa0사회환경 조성    지원대상  \xa0신청일 기준 1년 이상 계속하여 서초구에   주민등록 되어 있는 아빠 육아휴직자 (대상자녀 서초구에 주민등록 된 경우)\xa0 \xa0    신청기간  \xa0육아휴직 시작일 이후 1개월부터 종료일 이후 12개월 이내\xa0    신청방법  \xa0○ 온라인 : 서초구청 홈페이지\xa0 \xa0\xa0 - 경로 : 서초구청 홈페이지 > 분야별정보 > 복지   > 영유아복지 > 아빠육아휴직장려금 신청 [바로가기]\xa0 \xa0○ 방문 : 동주민센터, 구 여성보육과 방문\xa0    구비서류  \xa0○ 고용센터 발행 육아휴직급여 지급결정 통지서   1부 \xa0○ 주민등록등본 1부(세대원 이름과 전입일자 포함, 모든 구성원   주민번호 뒷자리 미포함)    지원내용  \xa0매월 30만원(지원기간 최대 1년)\xa0     '

In [28]:
# 해당 문의가 어떤 분류에 해당되는지 알기 위해 분류를 수집
def get_desc(html):
    tables = html.select("table")
    table = pd.read_html(str(tables[-1]))[0]
    table_0 = table[[0, 1]]
    table_1 = table[[2, 3]]
    table_1.columns = [0, 1]
    table_desc = pd.concat([table_0, table_1]).set_index(0).T
    return table_desc

get_desc(html)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류
1,다산콜센터,120다산콜재단,D0000043274645,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지


## 내용 수집 함수 만들기

In [29]:
def get_view_page(view_no):
    url = f"https://opengov.seoul.go.kr/civilappeal/view/?nid={view_no}"
    reponse = requests.get(url)
    html = bs(reponse.text, "lxml")
    time.sleep(0.1)
    try:
        content = html.select("div.line-all")[0].get_text()
        df_desc = get_desc(html)
        df_desc["내용"] = content
        df_desc["view_no"] = view_no
        return df_desc
    except:
        print(view_no)
        return pd.DataFrame()

In [30]:
# 오류가 나지 않는지 확인합니다.
get_view_page(view_no=21780668)

21780668


""


## 전체 내용 가져오기
* <font color="red">실습을 위해 위에서 일부 데이터만 가져왔습니다.</font>
* 전체 데이터 수집시에서 전체 수집한 데이터로 수집을 진행하면 됩니다.
* 전체 데이터 수집시에는 서버에 부담이 가지 않도록 time.sleep()을 통해 간격을 두고 수집해 주세요.

In [31]:
# tqdm.notebook 을 통해 수집 진행상태를 확인합니다.
from tqdm.notebook import tqdm
tqdm.pandas()

view = df["view_no"].progress_apply(get_view_page)

  0%|          | 0/30 [00:00<?, ?it/s]

## 수집한 내용 확인하기

In [40]:
# 일부 데이터만 슬라이싱 하기
view[:3]

0    0  원본시스템  작성자(책임자)            관리번호            ...
1    0  원본시스템  작성자(책임자)            관리번호     제공부서   ...
2    0  원본시스템  작성자(책임자)            관리번호            ...
Name: view_no, dtype: object

In [41]:
# 하나만 가져오면 데이터프레임 형태로 되어 있음을 확인
view[0]

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,view_no
1,다산콜센터,120다산콜재단,D0000043274645,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지,\n 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경제적 부담...,23522464


## 하나로 병합하기

In [42]:
# 수집한 내용을 tolist() 를 통해 리스트로 변환 후 저장
df_view = pd.concat(view.tolist())

In [45]:
# 기존 데이터와 병합, 내용이 함께 수집된 것을 확인
df_detail = df.merge(df_view.drop("생산일", axis=1), on=["view_no"])
df_detail.head()

,번호,제목,생산일,조회수,view_no,원본시스템,작성자(책임자),관리번호,제공부서,분류,내용
0,2645,아빠 육아휴직 장려금,2021-08-17,144,23522464,다산콜센터,120다산콜재단,D0000043274645,서울특별시 서초구 주민생활국 여성보육과,복지,\n 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경제적 부담...
1,2644,[서울산업진흥원] 서울메이드란?,2021-06-29,166,23194045,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,경제,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...
2,2643,(강북구) 정비중,2021-06-02,215,23032485,다산콜센터,120다산콜재단,D0000042693048,서울특별시 강북구 생활복지국 청소행정과,환경,\n 업무개요 투명 폐트병을 교환.보상하므로 수거율을 높이고 폐기물을 감량...
3,2642,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,288,22904492,다산콜센터,120다산콜재단,D0000042557522,서울특별시 광진구 복지국 가정복지과,복지,\n 업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄...
4,2641,마포 뇌병변장애인 비전센터,2021-03-12,441,22477798,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,복지,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...


## 사용할 컬럼만 남기기

In [46]:
df_detail.columns

Index(['번호', '제목', '생산일', '조회수', 'view_no', '원본시스템', '작성자(책임자)', '관리번호',
       '제공부서', '분류', '내용'],
      dtype='object')

In [48]:
# 제목 뒤에 문서가 보이도록 컬럼의 순서를 조정합니다.
# 사용할 컬럼만 인덱싱 합니다.
# 다음의 순서가 되게 합니다. '번호', '분류', '제목', '내용', 'view_no'
df_detail = df_detail[['번호', '분류', '제목', '내용', 'view_no']]

## 파일로 저장하고 확인하기

<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [49]:
# 저장할 파일명
file_name = "seoul-120-sample.csv"

In [53]:
# csv 파일로 저장하기
df_detail.to_csv(file_name, index=False)

In [52]:
# csv 파일로 읽어오기
pd.read_csv(file_name)

,번호,분류,제목,내용,view_no
0,2645,복지,아빠 육아휴직 장려금,\n 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경제적 부담...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...,23194045
2,2643,환경,(강북구) 정비중,\n 업무개요 투명 폐트병을 교환.보상하므로 수거율을 높이고 폐기물을 감량...,23032485
3,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",\n 업무개요 교통약자인 임산부와 영아가정 주민이 병원 진료를 받으러 갈 떄...,22904492
4,2641,복지,마포 뇌병변장애인 비전센터,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...,22477798
5,2640,행정,2021년도 중1·고1 신입생 입학준비금 지원,"\n 업무개요 서울시는 전국 최초로 2021년도부터 25개 자치구, 교육청과...",22227896
6,2639,복지,위드유 서울 직장 성희롱.성폭력 예방센터,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...,21212235
7,2638,경제,"[성동구] 반려동물 ""무료상담실"" 운영 안내","\n▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ...",21095911
8,2637,복지,서울시 '우리동네꼼꼼육아정보' 책자,\n서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰...,18955676
9,2636,교통,반반택시 (심야시간 택시 동승앱),\n 업무개요 ○ 심야 택시수요 대비 택시공급 부족으로 발생하는 승차난 해...,18409818
